In [ ]:
%matplotlib inline

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import pickle

# import stan interface
import pystan


In [ ]:
def compile_model(filename, model_name=None, **kwargs):
    """ Code taken from: https://github.com/avehtari/BDA_py_demos/blob/master/utilities_and_data/stan_utility.py
    
        This will automatically cache models - great if you're just running a
        script on the command line.
        See http://pystan.readthedocs.io/en/latest/avoiding_recompilation.html
    """
    from hashlib import md5

    with open(filename) as f:
        model_code = f.read()
        code_hash = md5(model_code.encode('ascii')).hexdigest()
        if model_name is None:
            cache_fn = 'cached-model-{}.pkl'.format(code_hash)
        else:
            cache_fn = 'cached-{}-{}.pkl'.format(model_name, code_hash)
        try:
            sm = pickle.load(open(cache_fn, 'rb'))
        except:
            sm = pystan.StanModel(model_code=model_code)
            with open(cache_fn, 'wb') as f:
                pickle.dump(sm, f)
        else:
            print("Using cached StanModel")
        return sm

In [ ]:
filename = "bernoulli-bandits-conjugate.stan"

with open(filename) as file:
    print(file.read())



In [ ]:
model = compile_model(filename)